### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_DENOVO_155/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_DENOVO_155


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks226 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_DENOVO_155/226_TROCAS7_Fusion_Apr2021_DENOVO_155_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks226))

print(peaks226.columns)

# These columns mess things up- get rid of them

del peaks226['Fraction']
del peaks226['Scan']
del peaks226['Source File']
del peaks226['Tag Length']
del peaks226['PTM']
del peaks226['tag (>=0%)']
del peaks226['mode']
del peaks226['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks226.columns = columns

mean_len = peaks226['length'].mean()
print(mean_len)

# look at the dataframe
peaks226.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 207
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
10.106280193236715


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,VLEGNEQFLNAAK,98,13,716.8762,2,71.89,1970000.0,1431.7358,1.4
1,LSSPATLNSR,98,10,523.2856,2,47.99,53600000.0,1044.5564,0.2
2,LSSPATLNSR,97,10,523.2861,2,47.49,53600000.0,1044.5564,1.1
3,LSSPATLNSR,96,10,523.2857,2,48.50,53600000.0,1044.5564,0.4
4,LSSPATLNSR,96,10,523.2861,2,46.98,53600000.0,1044.5564,1.3


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [9]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks226['A'] = peaks226['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks226['C'] = peaks226['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks226['D'] = peaks226['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks226['E'] = peaks226['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks226['F'] = peaks226['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks226['G'] = peaks226['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks226['H'] = peaks226['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks226 output, there will be no isoleucines (they're lumped in with leucines)
peaks226['I'] = peaks226['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks226['K'] = peaks226['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks226['L'] = peaks226['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks226['M'] = peaks226['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks226['N'] = peaks226['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks226['P'] = peaks226['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks226['Q'] = peaks226['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks226['R'] = peaks226['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks226['S'] = peaks226['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks226['T'] = peaks226['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks226['V'] = peaks226['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks226['W'] = peaks226['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks226['Y'] = peaks226['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks226['c-carb'] = peaks226['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks226['m-oxid'] = peaks226['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks226['n-deam'] = peaks226['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks226['q-deam'] = peaks226['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks226['stripped_peptide'] = peaks226['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks226['stripped_length'] = peaks226['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks226['ptm-total'] = peaks226['c-carb'] + peaks226['m-oxid'] + peaks226['n-deam'] + peaks226['q-deam']

# calculate NAAF numerator for each peptide k
peaks226['NAAF_num.'] = peaks226['Area'] / peaks226['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50.csv")

# check out the results
peaks226.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,VLEGNEQFLNAAK,98,13,716.8762,2,71.89,1970000.0,1431.7358,1.4,2,...,0,0,0,0,0,0,VLEGNEQFLNAAK,13,0,1.515385e+05
1,LSSPATLNSR,98,10,523.2856,2,47.99,53600000.0,1044.5564,0.2,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.360000e+06
2,LSSPATLNSR,97,10,523.2861,2,47.49,53600000.0,1044.5564,1.1,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.360000e+06
3,LSSPATLNSR,96,10,523.2857,2,48.50,53600000.0,1044.5564,0.4,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.360000e+06
4,LSSPATLNSR,96,10,523.2861,2,46.98,53600000.0,1044.5564,1.3,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,5.360000e+06


In [10]:
# keep only stripped peptide I/L and NAAF
dn_226 = peaks226[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_226.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/226_MN_T00_GD_DN50_ILnaafs.csv")

dn_226.head()

,stripped_peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,1970000.0,1.515385e+05
1,LSSPATLNSR,53600000.0,5.360000e+06
2,LSSPATLNSR,53600000.0,5.360000e+06
3,LSSPATLNSR,53600000.0,5.360000e+06
4,LSSPATLNSR,53600000.0,5.360000e+06


In [11]:
# made a new dataframe that contains the suMN of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks226['A'].sum(),
        'C': peaks226['C'].sum(),
        'D': peaks226['D'].sum(),
        'E': peaks226['E'].sum(),
        'F': peaks226['F'].sum(),
        'G': peaks226['G'].sum(),
        'H': peaks226['H'].sum(),
        'I': peaks226['I'].sum(),
        'K': peaks226['K'].sum(),
        'L': peaks226['L'].sum(),
        'M': peaks226['M'].sum(),
        'N': peaks226['N'].sum(),
        'P': peaks226['P'].sum(),
        'Q': peaks226['Q'].sum(),
        'R': peaks226['R'].sum(),
        'S': peaks226['S'].sum(),
        'T': peaks226['T'].sum(),
        'V': peaks226['V'].sum(),
        'W': peaks226['W'].sum(),
        'Y': peaks226['Y'].sum(),
        'c-carb': peaks226['c-carb'].sum(),
        'm-oxid': peaks226['m-oxid'].sum(),
        'n-deam': peaks226['n-deam'].sum(),
        'q-deam': peaks226['q-deam'].sum(),
        'Total area': peaks226['Area'].sum(),
        'Total length': peaks226['stripped_length'].sum()
       }

totalpeaks226 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks226['% C w/ carb'] = totalpeaks226['c-carb'] / totalpeaks226['C'] 

# calculate percentage of M's that are oxidized
totalpeaks226['% M w/ oxid'] = totalpeaks226['m-oxid'] / totalpeaks226['M'] 

# calculate percentage of N's that are deamidated
totalpeaks226['% N w/ deam'] = totalpeaks226['n-deam'] / totalpeaks226['N'] 

# calculate percentage of N's that are deamidated
totalpeaks226['% Q w/ deam'] = totalpeaks226['q-deam'] / totalpeaks226['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks226['NAAF denom.'] = totalpeaks226['Total area'] / totalpeaks226['Total length']

# write modified dataframe to new txt file
totalpeaks226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN//226_MN_T00_GD_DN50_totals.csv")

totalpeaks226.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,115,40,44,47,93,98,143,0,183,286,...,23,10,5,2.261064e+09,2004,1.0,0.190083,0.09009,0.121951,1.128276e+06


In [12]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 1.128276e+06

# use NAAF >50% ALC to get NAAF factor
peaks226['NAAF factor'] = (peaks226['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksAAPTM_226 = peaks226[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_226['A-NAAF50'] = peaksAAPTM_226['A'] * peaks226['NAAF factor']
peaksAAPTM_226['C-NAAF50'] = peaksAAPTM_226['C'] * peaks226['NAAF factor']
peaksAAPTM_226['D-NAAF50'] = peaksAAPTM_226['D'] * peaks226['NAAF factor']
peaksAAPTM_226['E-NAAF50'] = peaksAAPTM_226['E'] * peaks226['NAAF factor']
peaksAAPTM_226['F-NAAF50'] = peaksAAPTM_226['F'] * peaks226['NAAF factor']
peaksAAPTM_226['G-NAAF50'] = peaksAAPTM_226['G'] * peaks226['NAAF factor']
peaksAAPTM_226['H-NAAF50'] = peaksAAPTM_226['H'] * peaks226['NAAF factor']
peaksAAPTM_226['I-NAAF50'] = peaksAAPTM_226['I'] * peaks226['NAAF factor']
peaksAAPTM_226['K-NAAF50'] = peaksAAPTM_226['K'] * peaks226['NAAF factor']
peaksAAPTM_226['L-NAAF50'] = peaksAAPTM_226['L'] * peaks226['NAAF factor']
peaksAAPTM_226['M-NAAF50'] = peaksAAPTM_226['M'] * peaks226['NAAF factor']
peaksAAPTM_226['N-NAAF50'] = peaksAAPTM_226['N'] * peaks226['NAAF factor']
peaksAAPTM_226['P-NAAF50'] = peaksAAPTM_226['P'] * peaks226['NAAF factor']
peaksAAPTM_226['Q-NAAF50'] = peaksAAPTM_226['Q'] * peaks226['NAAF factor']
peaksAAPTM_226['R-NAAF50'] = peaksAAPTM_226['R'] * peaks226['NAAF factor']
peaksAAPTM_226['S-NAAF50'] = peaksAAPTM_226['S'] * peaks226['NAAF factor']
peaksAAPTM_226['T-NAAF50'] = peaksAAPTM_226['T'] * peaks226['NAAF factor']
peaksAAPTM_226['V-NAAF50'] = peaksAAPTM_226['V'] * peaks226['NAAF factor']
peaksAAPTM_226['W-NAAF50'] = peaksAAPTM_226['W'] * peaks226['NAAF factor']
peaksAAPTM_226['Y-NAAF50'] = peaksAAPTM_226['Y'] * peaks226['NAAF factor']

# multiply the NAAF50 factor by the PTMN normalize its abundance by peak area and peptide length

peaksAAPTM_226['ccarb-NAAF50'] = peaksAAPTM_226['c-carb'] * peaksAAPTM_226['NAAF factor']
peaksAAPTM_226['moxid-NAAF50'] = peaksAAPTM_226['m-oxid'] * peaksAAPTM_226['NAAF factor']
peaksAAPTM_226['ndeam-NAAF50'] = peaksAAPTM_226['n-deam'] * peaksAAPTM_226['NAAF factor']
peaksAAPTM_226['qdeam-NAAF50'] = peaksAAPTM_226['q-deam'] * peaksAAPTM_226['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_naaf.csv")

peaksAAPTM_226.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,VLEGNEQFLNAAK,0.134310,2,0,0,2,1,1,0,1,...,0.000000,0.000000,0.000000,0.13431,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,4.750611,1,0,0,0,0,0,0,0,...,4.750611,14.251832,4.750611,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,4.750611,1,0,0,0,0,0,0,0,...,4.750611,14.251832,4.750611,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,LSSPATLNSR,4.750611,1,0,0,0,0,0,0,0,...,4.750611,14.251832,4.750611,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,LSSPATLNSR,4.750611,1,0,0,0,0,0,0,0,...,4.750611,14.251832,4.750611,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMN

index = ['sample total']

data = {'NAAF': peaksAAPTM_226['NAAF factor'].sum(),
        'A': peaksAAPTM_226['A-NAAF50'].sum(),
        'C': peaksAAPTM_226['C-NAAF50'].sum(),
        'D': peaksAAPTM_226['D-NAAF50'].sum(),
        'E': peaksAAPTM_226['E-NAAF50'].sum(),
        'F': peaksAAPTM_226['F-NAAF50'].sum(),
        'G': peaksAAPTM_226['G-NAAF50'].sum(),
        'H': peaksAAPTM_226['H-NAAF50'].sum(),
        'I': peaksAAPTM_226['I-NAAF50'].sum(),
        'K': peaksAAPTM_226['K-NAAF50'].sum(),
        'L': peaksAAPTM_226['L-NAAF50'].sum(),
        'M': peaksAAPTM_226['M-NAAF50'].sum(),
        'N': peaksAAPTM_226['N-NAAF50'].sum(),
        'P': peaksAAPTM_226['P-NAAF50'].sum(),
        'Q': peaksAAPTM_226['Q-NAAF50'].sum(),
        'R': peaksAAPTM_226['R-NAAF50'].sum(),
        'S': peaksAAPTM_226['S-NAAF50'].sum(),
        'T': peaksAAPTM_226['T-NAAF50'].sum(),
        'V': peaksAAPTM_226['V-NAAF50'].sum(),
        'W': peaksAAPTM_226['W-NAAF50'].sum(),
        'Y': peaksAAPTM_226['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_226['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_226['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_226['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_226['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 1.128276e+06

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,237.712643,138.93727,3.679796,3.948846,2.994405,19.954045,9.790066,39.104299,0.0,23.623271,...,12.170707,3.679796,2.630691,0.500297,0.579595,1.0,0.106331,0.016143,0.177867,0.000211


## Export stripped peptides >50% ALC

In [14]:
##### keep only stripped peptide column 
pep50 = peaks226[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/226_MN_T00_GD_DN/226_MN_T00_GD_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks226['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks226[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 207
# nonredundant stripped Peaks peptides >50% ALC 127
average peptide length Peaks peptides >50% ALC 9.681159420289855
# redundant Peaks peptides >50% ALC 207
# nonredundant Peaks peptides 127


,stripped_peptide
0,VLEGNEQFLNAAK
1,LSSPATLNSR
2,LSSPATLNSR
3,LSSPATLNSR
4,LSSPATLNSR
